In [25]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
%matplotlib inline

In [26]:
import datetime 

interventions_df = pd.read_csv('data/donneesouvertes-interventions-sim.csv')
interventions_df['CREATION_DATE_TIME'] = interventions_df['CREATION_DATE_TIME'].astype('datetime64[ns]')  
interventions_df.head()

,INCIDENT_NBR,CREATION_DATE_TIME,INCIDENT_TYPE_DESC,DESCRIPTION_GROUPE,CASERNE,NOM_VILLE,NOM_ARROND,DIVISION,NOMBRE_UNITES,CIV,MTM8_X,MTM8_Y,LONGITUDE,LATITUDE
0,1168,2015-01-03 20:56:02,Premier répondant,1-REPOND,33,Montréal,Sud-Ouest,8,1.0,1.0,297283.0,5035433.1,-73.596117,45.458786
1,1171,2015-01-03 21:03:52,Alarme privé ou locale,Alarmes-incendies,22,Montréal,Saint-Léonard,4,4.0,1.0,299180.0,5049284.2,-73.572020,45.583442
2,1172,2015-01-03 21:07:00,Premier répondant,1-REPOND,9,Montréal,Villeray / St-Michel / Parc Extension,4,1.0,1.0,297259.6,5046768.2,-73.596589,45.560784
3,1177,2015-01-03 21:41:46,Premier répondant,1-REPOND,37,Montréal,Villeray / St-Michel / Parc Extension,4,1.0,1.0,294738.3,5044817.2,-73.628847,45.543197
4,1178,2015-01-03 21:43:32,Aliments surchauffés,SANS FEU,16,Montréal,Plateau Mont-Royal,11,5.0,1.0,298505.7,5042556.1,-73.580573,45.522895


In [27]:
interventions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 538766 entries, 0 to 538765
Data columns (total 14 columns):
INCIDENT_NBR          538766 non-null int64
CREATION_DATE_TIME    538766 non-null datetime64[ns]
INCIDENT_TYPE_DESC    538766 non-null object
DESCRIPTION_GROUPE    538760 non-null object
CASERNE               538766 non-null int64
NOM_VILLE             538766 non-null object
NOM_ARROND            538766 non-null object
DIVISION              538766 non-null int64
NOMBRE_UNITES         538633 non-null float64
CIV                   479899 non-null float64
MTM8_X                538766 non-null float64
MTM8_Y                538766 non-null float64
LONGITUDE             538766 non-null float64
LATITUDE              538766 non-null float64
dtypes: datetime64[ns](1), float64(6), int64(3), object(4)
memory usage: 57.5+ MB


In [28]:
interventions_df['NOM_ARROND'] = np.where(interventions_df['NOM_ARROND'] == 'Indéterminé', interventions_df['NOM_VILLE'], interventions_df['NOM_ARROND']) 
interventions_df.head()

,INCIDENT_NBR,CREATION_DATE_TIME,INCIDENT_TYPE_DESC,DESCRIPTION_GROUPE,CASERNE,NOM_VILLE,NOM_ARROND,DIVISION,NOMBRE_UNITES,CIV,MTM8_X,MTM8_Y,LONGITUDE,LATITUDE
0,1168,2015-01-03 20:56:02,Premier répondant,1-REPOND,33,Montréal,Sud-Ouest,8,1.0,1.0,297283.0,5035433.1,-73.596117,45.458786
1,1171,2015-01-03 21:03:52,Alarme privé ou locale,Alarmes-incendies,22,Montréal,Saint-Léonard,4,4.0,1.0,299180.0,5049284.2,-73.572020,45.583442
2,1172,2015-01-03 21:07:00,Premier répondant,1-REPOND,9,Montréal,Villeray / St-Michel / Parc Extension,4,1.0,1.0,297259.6,5046768.2,-73.596589,45.560784
3,1177,2015-01-03 21:41:46,Premier répondant,1-REPOND,37,Montréal,Villeray / St-Michel / Parc Extension,4,1.0,1.0,294738.3,5044817.2,-73.628847,45.543197
4,1178,2015-01-03 21:43:32,Aliments surchauffés,SANS FEU,16,Montréal,Plateau Mont-Royal,11,5.0,1.0,298505.7,5042556.1,-73.580573,45.522895


In [29]:
fire_types = [
'10-22 avec feu',
'Feu de champ *',
'Ac.vic. feu train/m√©tro F7/DES',
'Acc. sans vict. av. feu - ext.',
'Acc. sans vict. av. feu-tunnel',
'Acc. s-vict. av. feu - pont',
'Acc. s-vict. av. feu - v.r.',
'Acc. s-vict. av. feu-b√¢timent',
'Acc. s-vict. feu - train/m√©tro',
'Acc. vict. feu - tunnel F7/MOU',
'Acc. vict. sfeu tunnel F7/MOU',
'Acc. victime feu - b√¢timent',
'Acc. victime feu - pont',
'Acc. victime feu - voie rapide',
'Acc. victime feu ext.',
'Acc. victime sfeu - b√¢timent',
'Acc. victime sfeu - pont',
'Acc. victime sfeu - voie rap.',
'D√©chets en feu',
'Feu / 2e Alerte',
'Feu / 3e Alerte',
'Feu / 4e Alerte',
'Feu / 5e Alerte',
'Feu / Agravation possible',
'Feu d\'auto (voie rapide)',
'Feu de b√¢timent',
'Feu de chemin√©e *',
'Feu de cuisson',
'Feu de nature √©lectrique',
'Feu de navire/b√¢teau',
'Feu de v√©hicule ext√©rieur',
'Feu de v√©hicule int√©rieur',
'Feu de v√©hicule sur pont',
'feu de v√©hicule tunnel F7/MOU',
'Feu v√©h. convoi (train/m√©tro)',
'M√©tro b√¢timent /10-22 avec feu',
'M√©tro b√¢timent /10-22 sans feu',
'Senteur de feu √† l\'ext√©rieur]']

fire_interventions = interventions_df[interventions_df['INCIDENT_TYPE_DESC'].isin(fire_types)]
fire_interventions.head()

,INCIDENT_NBR,CREATION_DATE_TIME,INCIDENT_TYPE_DESC,DESCRIPTION_GROUPE,CASERNE,NOM_VILLE,NOM_ARROND,DIVISION,NOMBRE_UNITES,CIV,MTM8_X,MTM8_Y,LONGITUDE,LATITUDE
14,1599,2015-01-04 15:35:19,Feu de champ *,AUTREFEU,53,Beaconsfield,Beaconsfield,2,1.0,1.0,276170.9,5031835.0,-73.865855,45.425863
29,1623,2015-01-04 15:59:59,Feu de champ *,AUTREFEU,66,Montréal,Verdun,8,1.0,1.0,298843.7,5035305.0,-73.576159,45.457649
61,2864,2015-01-06 23:01:06,10-22 avec feu,INCENDIE,19,Montréal,Ville-Marie,6,6.0,1.0,301176.3,5043239.1,-73.546392,45.529059
223,2905,2015-01-07 02:37:03,10-22 avec feu,INCENDIE,73,Montréal,Saint-Laurent,3,7.0,1.0,291393.6,5042740.1,-73.671621,45.524449
252,3722,2015-01-08 21:18:19,Feu de champ *,AUTREFEU,29,Montréal,Rosemont / Petite-Patrie,7,1.0,1.0,298634.7,5046099.2,-73.578967,45.554778


In [30]:
fire_interventions_pivot = fire_interventions.pivot_table(values='INCIDENT_NBR', index='NOM_ARROND', columns='INCIDENT_TYPE_DESC', aggfunc=pd.Series.nunique, fill_value = 0)
fire_interventions_pivot.shape

(34, 14)

In [31]:
fire_interventions_pivot.head(10)


INCIDENT_TYPE_DESC,10-22 avec feu,Acc. s-vict. av. feu - pont,Acc. s-vict. av. feu - v.r.,Acc. sans vict. av. feu - ext.,Acc. vict. sfeu tunnel F7/MOU,Acc. victime sfeu - pont,Acc. victime sfeu - voie rap.,Feu / 2e Alerte,Feu / 3e Alerte,Feu / 4e Alerte,Feu / 5e Alerte,Feu / Agravation possible,Feu d'auto (voie rapide),Feu de champ *
NOM_ARROND,,,,,,,,,,,,,,
Ahuntsic / Cartierville,178,1,0,4,0,0,1,6,5,1,1,3,15,380
Anjou,51,0,0,2,0,0,0,0,1,1,1,0,19,79
Baie d'Urfé,10,0,0,0,0,0,0,1,0,0,0,0,3,10
Beaconsfield,13,0,0,0,0,0,2,0,0,0,0,0,5,26
Côte St-Luc,34,0,0,1,0,0,0,4,1,0,0,3,0,55
Côte-des-Neiges / Notre-Dame-de-Grâce,285,0,2,3,0,0,0,20,5,3,2,4,10,297
Dollard-des-Ormeaux,51,0,0,1,0,0,0,1,0,0,0,3,0,61
Dorval,42,0,0,0,0,0,1,1,0,0,0,1,11,59
Hampstead,7,0,0,1,0,0,0,0,0,0,0,0,0,9


In [32]:
incident_type_df = pd.DataFrame(fire_interventions_pivot.to_records())
incident_type_df.head()

,NOM_ARROND,10-22 avec feu,Acc. s-vict. av. feu - pont,Acc. s-vict. av. feu - v.r.,Acc. sans vict. av. feu - ext.,Acc. vict. sfeu tunnel F7/MOU,Acc. victime sfeu - pont,Acc. victime sfeu - voie rap.,Feu / 2e Alerte,Feu / 3e Alerte,Feu / 4e Alerte,Feu / 5e Alerte,Feu / Agravation possible,Feu d'auto (voie rapide),Feu de champ *
0,Ahuntsic / Cartierville,178,1,0,4,0,0,1,6,5,1,1,3,15,380
1,Anjou,51,0,0,2,0,0,0,0,1,1,1,0,19,79
2,Baie d'Urfé,10,0,0,0,0,0,0,1,0,0,0,0,3,10
3,Beaconsfield,13,0,0,0,0,0,2,0,0,0,0,0,5,26
4,Côte St-Luc,34,0,0,1,0,0,0,4,1,0,0,3,0,55


In [33]:
fire_dict = {
'NOM_ARROND': 'NOM_ARROND',
'10-22 avec feu':"Total_Fires",
'Feu de champ *':"Total_Fires",
'Ac.vic. feu train/m√©tro F7/DES':"Total_Fires",
'Acc. sans vict. av. feu - ext.':"Total_Fires",
'Acc. sans vict. av. feu-tunnel':"Total_Fires",
'Acc. s-vict. av. feu - pont':"Total_Fires",
'Acc. s-vict. av. feu - v.r.':"Total_Fires",
'Acc. s-vict. av. feu-b√¢timent':"Total_Fires",
'Acc. s-vict. feu - train/m√©tro':"Total_Fires",
'Acc. vict. feu - tunnel F7/MOU':"Total_Fires",
'Acc. vict. sfeu tunnel F7/MOU':"Total_Fires",
'Acc. victime feu - b√¢timent':"Total_Fires",
'Acc. victime feu - pont':"Total_Fires",
'Acc. victime feu - voie rapide':"Total_Fires",
'Acc. victime feu ext.':"Total_Fires",
'Acc. victime sfeu - b√¢timent':"Total_Fires",
'Acc. victime sfeu - pont':"Total_Fires",
'Acc. victime sfeu - voie rap.':"Total_Fires",
'D√©chets en feu':"Total_Fires",
'Feu / 2e Alerte':"Total_Fires",
'Feu / 3e Alerte':"Total_Fires",
'Feu / 4e Alerte':"Total_Fires",
'Feu / 5e Alerte':"Total_Fires",
'Feu / Agravation possible':"Total_Fires",
'Feu d\'auto (voie rapide)':"Total_Fires",
'Feu de b√¢timent':"Total_Fires",
'Feu de chemin√©e *':"Total_Fires",
'Feu de cuisson':"Total_Fires",
'Feu de nature √©lectrique':"Total_Fires",
'Feu de navire/b√¢teau':"Total_Fires",
'Feu de v√©hicule ext√©rieur':"Total_Fires",
'Feu de v√©hicule int√©rieur':"Total_Fires",
'Feu de v√©hicule sur pont':"Total_Fires",
'feu de v√©hicule tunnel F7/MOU':"Total_Fires",
'Feu v√©h. convoi (train/m√©tro)':"Total_Fires",
'M√©tro b√¢timent /10-22 avec feu':"Total_Fires",
'M√©tro b√¢timent /10-22 sans feu':"Total_Fires",
'Senteur de feu √† l\'ext√©rieur]':"Total_Fires"}

fire_interventions_total = incident_type_df.groupby(fire_dict, axis = 1).sum()
fire_interventions_total.head(20)

,NOM_ARROND,Total_Fires
0,Ahuntsic / Cartierville,595
1,Anjou,154
2,Baie d'Urfé,24
3,Beaconsfield,46
4,Côte St-Luc,98
5,Côte-des-Neiges / Notre-Dame-de-Grâce,631
6,Dollard-des-Ormeaux,117
7,Dorval,115
8,Hampstead,17
9,Ile Dorval,2


In [34]:
construction_year_df = pd.read_csv('data/uniteevaluationfonciere.csv')
construction_year_df = construction_year_df[['CIVIQUE_DEBUT','NOM_RUE','NOMBRE_LOGEMENT','ANNEE_CONSTRUCTION','NO_ARROND_ILE_CUM']]
construction_year_df.head()

,CIVIQUE_DEBUT,NOM_RUE,NOMBRE_LOGEMENT,ANNEE_CONSTRUCTION,NO_ARROND_ILE_CUM
0,2544,avenue Fletcher (MTL),1.0,2012.0,REM22
1,1908,boulevard Guy-Bouchard (LSL),1.0,1997.0,REM17
2,72,15e Rue (ROX),1.0,1958.0,REM31
3,105,avenue Parkdale (PCL),1.0,1974.0,REM99
4,45,13e Avenue (ROX),1.0,1955.0,REM31


In [35]:
borough_map = {
'REM05': 'Outremont',
'REM09': 'Anjou',
'REM12': 'Verdun',
'REM14': 'Saint-Léonard',
'REM15': 'Saint-Laurent',
'REM16': 'Montréal-Nord',
'REM17': 'Lasalle',
'REM19': 'Ville-Marie',
'REM20': 'Sud-Ouest',
'REM21': 'Plateau Mont-Royal',
'REM22': 'Mercier / Hochelaga-Maisonneuve',
'REM23': 'Ahuntsic / Cartierville',
'REM24': 'Rosemont / Petite-Patrie',
'REM25': 'Villeray / St-Michel / Parc Extension',
'REM27': 'Lachine',
'REM31': 'Pierrefonds / Roxboro',
'REM32': 'L\'Ile-Bizard / Ste-Geneviève',
'REM33': 'Rivière-des-Prairies / Pointe-aux-Trembles',
'REM34': 'Côte-des-Neiges / Notre-Dame-de-Grâce'
}

construction_year_df["NOM_ARROND"] = construction_year_df["NO_ARROND_ILE_CUM"].map(borough_map)
construction_year_df.head(30)

,CIVIQUE_DEBUT,NOM_RUE,NOMBRE_LOGEMENT,ANNEE_CONSTRUCTION,NO_ARROND_ILE_CUM,NOM_ARROND
0,2544,avenue Fletcher (MTL),1.0,2012.0,REM22,Mercier / Hochelaga-Maisonneuve
1,1908,boulevard Guy-Bouchard (LSL),1.0,1997.0,REM17,Lasalle
2,72,15e Rue (ROX),1.0,1958.0,REM31,Pierrefonds / Roxboro
3,105,avenue Parkdale (PCL),1.0,1974.0,REM99,NaN
4,45,13e Avenue (ROX),1.0,1955.0,REM31,Pierrefonds / Roxboro
5,150,avenue Tremont (DVL),1.0,1985.0,REM99,NaN
6,70,avenue Courcelette (OUT),1.0,1923.0,REM05,Outremont
7,2301,rue Saint-Patrick (LSL+MTL),NaN,2015.0,REM20,Sud-Ouest
8,6380,21e Avenue (MTL),1.0,2003.0,REM24,Rosemont / Petite-Patrie
9,2459,chemin Laval (SLR),3.0,1959.0,REM15,Saint-Laurent


In [36]:
construction_year_df[['Street', 'Trigram']] = construction_year_df['NOM_RUE'].str.split('(',expand=True)
construction_year_df.drop('Street', axis=1, inplace=True)
construction_year_df['Trigram'] = construction_year_df['Trigram'].str.rstrip(')')
construction_year_df.head()

,CIVIQUE_DEBUT,NOM_RUE,NOMBRE_LOGEMENT,ANNEE_CONSTRUCTION,NO_ARROND_ILE_CUM,NOM_ARROND,Trigram
0,2544,avenue Fletcher (MTL),1.0,2012.0,REM22,Mercier / Hochelaga-Maisonneuve,MTL
1,1908,boulevard Guy-Bouchard (LSL),1.0,1997.0,REM17,Lasalle,LSL
2,72,15e Rue (ROX),1.0,1958.0,REM31,Pierrefonds / Roxboro,ROX
3,105,avenue Parkdale (PCL),1.0,1974.0,REM99,NaN,PCL
4,45,13e Avenue (ROX),1.0,1955.0,REM31,Pierrefonds / Roxboro,ROX


In [38]:
construction_year_df_99 = construction_year_df[construction_year_df['NO_ARROND_ILE_CUM'] == 'REM99']
construction_year_df = construction_year_df[construction_year_df['NO_ARROND_ILE_CUM'] != 'REM99']

other_borough_map = {
    'PCL': 'Pointe-Claire', 
    'HMS': 'Hampstead',
    'SNV': 'Senneville',
    'DVL': 'Dorval', 
    'MTE': 'Montréal-Est',
    'MTR': 'Mont-Royal', 
    'CSL': 'Côte St-Luc', 
    'MTL+WMT': 'Westmount', 
    'BCF': 'Beaconsfield', 
    'KRK': 'Kirkland',
    'DDO': 'Dollard-des-Ormeaux',
    'CSL+MTL': 'Côte St-Luc',  
    'MTO': 'Montréal-Ouest', 
    'WMT': 'Westmount', 
    'DDO+PFD': 'Dollard-des-Ormeaux',
    'BDU': 'Baie d\'Urfé', 
    'MTL+MTR':'Mont-Royal',
    'SAB': 'Ste-Anne-de-Bellevue'
}

construction_year_df_99["NOM_ARROND"] = construction_year_df_99.Trigram.map(other_borough_map)
construction_year_df = construction_year_df.append(construction_year_df_99)
construction_year_df.drop(['Trigram', 'NO_ARROND_ILE_CUM', 'CIVIQUE_DEBUT', 'NOM_RUE', 'NOMBRE_LOGEMENT'], axis=1, inplace=True)
construction_year_df.head()

,ANNEE_CONSTRUCTION,NOM_ARROND
0,2012.0,Mercier / Hochelaga-Maisonneuve
1,1997.0,Lasalle
2,1958.0,Pierrefonds / Roxboro
4,1955.0,Pierrefonds / Roxboro
6,1923.0,Outremont


In [39]:
construction_year_group = construction_year_df.groupby("NOM_ARROND", as_index=True)['ANNEE_CONSTRUCTION'].mean()
construction_year_group.head()

NOM_ARROND
Ahuntsic / Cartierville                  1969.434241
Anjou                                    1983.330827
Côte-des-Neiges / Notre-Dame-de-Grâce    1958.444416
L'Ile-Bizard / Ste-Geneviève             1981.385212
Lachine                                  1966.610420
Name: ANNEE_CONSTRUCTION, dtype: float64

In [40]:
construction_year_group_df = pd.DataFrame(construction_year_group)
construction_year_group_df.head()

,ANNEE_CONSTRUCTION
NOM_ARROND,
Ahuntsic / Cartierville,1969.434241
Anjou,1983.330827
Côte-des-Neiges / Notre-Dame-de-Grâce,1958.444416
L'Ile-Bizard / Ste-Geneviève,1981.385212
Lachine,1966.610420


In [41]:
fire_full_df = pd.merge(fire_interventions_total, construction_year_group_df, on = 'NOM_ARROND', how='left')
fire_full_df.head(35)

,NOM_ARROND,Total_Fires,ANNEE_CONSTRUCTION
0,Ahuntsic / Cartierville,595,1969.434241
1,Anjou,154,1983.330827
2,Baie d'Urfé,24,NaN
3,Beaconsfield,46,NaN
4,Côte St-Luc,98,NaN
5,Côte-des-Neiges / Notre-Dame-de-Grâce,631,1958.444416
6,Dollard-des-Ormeaux,117,NaN
7,Dorval,115,NaN
8,Hampstead,17,NaN
9,Ile Dorval,2,NaN


In [55]:
socio_demo_df = pd.read_csv('data/socio-demo.csv')
socio_demo_df.head(10)

,NOM_ARROND,Densité,Male,Female,Âge moyen de la population,Âge médian de la population,Couple with kids,Couple without kids,5 plus families,4 member families,...,Unofficial Language,Immigrants,Non-immigrants,"Population selon le plus haut certificat, diplôme ou grade",No certificate,Taux d'emploi (%),Travailleurs autonomes,Revenu avant impôt en 2015 de la population de 15 ans et plus,Taux d'emploi (%).1,Taux de chômage (%)
0,Ahuntsic / Cartierville,5556.498344,47.98%,52.02%,4160.00%,3990.00%,46.30%,33.60%,9.60%,20.70%,...,41.60%,40.40%,57.90%,106080,18%,56.0,11.1%,106075,56%,10%
1,Anjou,3128.362573,46.38%,53.62%,4390.00%,4380.00%,42.00%,37.90%,9.10%,19.90%,...,33.60%,33.50%,65.70%,34455,20%,52.4,9.4%,34455,52%,10%
2,Côte-des-Neiges / Notre-Dame-de-Grâce,7766.791045,47.82%,52.18%,3870.00%,3610.00%,45.30%,32.80%,9.00%,19.60%,...,43.80%,46.70%,46.90%,136325,12%,56.9,11.9%,136325,57%,11%
3,Lachine,2510.665914,48.02%,51.98%,4180.00%,4020.00%,41.40%,34.40%,7.00%,19.30%,...,19.60%,22.50%,76.70%,35315,21%,58.9,8.7%,35315,59%,9%
4,Lasalle,4723.601721,47.41%,52.59%,4220.00%,4160.00%,42.10%,34.20%,8.00%,18.80%,...,36.70%,35.90%,62.40%,62680,20%,54.5,10.4%,62675,55%,10%
5,Plateau Mont-Royal,12792.127920,50.89%,49.11%,3740.00%,3360.00%,32.30%,51.70%,4.50%,13.00%,...,20.90%,24.00%,65.10%,89955,8%,67.1,15.4%,89955,67%,8%
6,Sud-Ouest,4984.119898,49.51%,50.49%,3890.00%,3570.00%,32.40%,43.70%,6.10%,13.80%,...,24.50%,25.10%,72.10%,65805,19%,61.0,10.9%,65810,61%,9%
7,L'Ile-Bizard / Ste-Geneviève,780.211864,48.86%,51.14%,4210.00%,4310.00%,50.80%,33.70%,8.60%,24.60%,...,26.60%,25.20%,73.90%,14855,13%,64.0,13.6%,14855,64%,7%
8,Mercier / Hochelaga-Maisonneuve,5353.168044,49.64%,50.36%,4120.00%,3850.00%,34.60%,42.10%,5.80%,14.90%,...,19.30%,21.00%,77.40%,113215,20%,59.4,8.7%,113215,59%,9%
9,Montréal-Nord,7622.986425,47.36%,52.64%,4120.00%,4000.00%,42.20%,27.50%,11.90%,19.00%,...,45.00%,41.70%,57.00%,66000,31%,49.1,8.8%,66005,49%,12%


In [56]:
fire_full_df = pd.merge(fire_full_df, socio_demo_df, on = 'NOM_ARROND', how='left')
fire_full_df.head(35)

,NOM_ARROND,Total_Fires,ANNEE_CONSTRUCTION,Densité_x,Male_x,Female_x,Âge moyen de la population_x,Âge médian de la population_x,Couple with kids_x,Couple without kids_x,...,Unofficial Language,Immigrants,Non-immigrants,"Population selon le plus haut certificat, diplôme ou grade",No certificate,Taux d'emploi (%),Travailleurs autonomes,Revenu avant impôt en 2015 de la population de 15 ans et plus,Taux d'emploi (%).1,Taux de chômage (%)
0,Ahuntsic / Cartierville,595,1969.434241,"5,556.5",48.0%,52.0%,41.6,39.9,46.3%,33.6%,...,41.60%,40.40%,57.90%,106080.0,18%,56.0,11.1%,106075.0,56%,10%
1,Anjou,154,1983.330827,"3,128.4",46.4%,53.6%,43.9,43.8,42.0%,37.9%,...,33.60%,33.50%,65.70%,34455.0,20%,52.4,9.4%,34455.0,52%,10%
2,Baie d'Urfé,24,NaN,635.8,48.4%,51.7%,45.3,47.9,51.3%,39.7%,...,23.60%,25.80%,71.30%,3115.0,7%,53.5,20.3%,3115.0,54%,7%
3,Beaconsfield,46,NaN,"1,759.9",48.7%,51.3%,41.7,44.3,54.7%,33.8%,...,21.40%,23.30%,75.90%,15595.0,8%,59.8,17.5%,15595.0,60%,6%
4,Côte St-Luc,98,NaN,"4,675.5",45.9%,54.1%,45.8,44.4,45.3%,38.5%,...,37.70%,44.80%,52.70%,25750.0,11%,50.8,19.3%,25750.0,51%,9%
5,Côte-des-Neiges / Notre-Dame-de-Grâce,631,1958.444416,"7,766.8",47.8%,52.2%,38.7,36.1,45.3%,32.8%,...,43.80%,46.70%,46.90%,136325.0,12%,56.9,11.9%,136325.0,57%,11%
6,Dollard-des-Ormeaux,117,NaN,"3,223.4",48.6%,51.4%,41.9,42.7,52.8%,31.2%,...,40.60%,39.60%,59.60%,40295.0,13%,59.6,12.7%,40295.0,60%,7%
7,Dorval,115,NaN,910.3,48.8%,51.2%,43.5,43.8,42.6%,40.0%,...,25.40%,25.90%,73.00%,16010.0,12%,63.3,10.2%,16010.0,63%,6%
8,Hampstead,17,NaN,"3,904.3",48.1%,51.9%,40.1,40.7,55.1%,32.5%,...,23.80%,28.40%,68.60%,5665.0,7%,62.0,29.6%,5665.0,62%,6%
9,Ile Dorval,2,NaN,26.1,N/D,N/D,N/D,N/D,N/D,N/D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
fire_full_no_boroughs_df = fire_full_df.drop('NOM_ARROND', axis=1)
fire_full_no_boroughs_df = scaler.fit_transform(fire_full_no_boroughs_df)
fire_full_no_boroughs_df.head(30)

ValueError: could not convert string to float: '5,556.5'

In [ ]:
#Finding optimal no. of clusters
from scipy.spatial.distance import cdist
clusters=range(1,10)
meanDistortions=[]
 
for k in clusters:
    model=KMeans(n_clusters=k)
    model.fit(clust_data)
    prediction=model.predict(clust_data)
    meanDistortions.append(sum(np.min(cdist(clust_data, model.cluster_centers_, 'euclidean'), axis=1)) / clust_data.shape[0])
 
#plt.cla()
plt.plot(clusters, meanDistortions, 'bx-')
plt.xlabel('k')
plt.ylabel('Average distortion')
plt.title('Selecting k with the Elbow Method')

In [ ]:
final_model=KMeans(8)
final_model.fit(clust_data)
prediction=final_model.predict(clust_data)
 
#Join predicted clusters back to raw data
incident_type_df["GROUP"] = prediction
print("Groups Assigned : \n")
incident_type_df[["GROUP","NOM_ARROND"]].sort_values('GROUP')

In [ ]:
gen = (x for x in incident_type_df if x not in "NOM_ARROND")

for column in gen:
    boxplot_field = column
    plt.figure()
    plt.suptitle(boxplot_field)
    plt.boxplot([incident_type_df[boxplot_field][incident_type_df.GROUP==0],
            incident_type_df[boxplot_field][incident_type_df.GROUP==1],
            incident_type_df[boxplot_field][incident_type_df.GROUP==2],
            incident_type_df[boxplot_field][incident_type_df.GROUP==3],
            incident_type_df[boxplot_field][incident_type_df.GROUP==4],
            incident_type_df[boxplot_field][incident_type_df.GROUP==5],
            incident_type_df[boxplot_field][incident_type_df.GROUP==6],
            incident_type_df[boxplot_field][incident_type_df.GROUP==7]],
            labels=('G1','G2','G3','G4','G5','G6','G7','G8'));

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

labelencoder = LabelEncoder()
arrond_df = pd.DataFrame(labelencoder.fit_transform(interventions_df['NOM_ARROND']))


onehotencoder = OneHotEncoder()
arrond_df = pd.DataFrame(onehotencoder.fit_transform(arrond_df).toarray())


In [ ]:
arrond_df.head()

In [ ]:
kmeans = KMeans(n_clusters = 20, n_init = 5, n_jobs = -1)
kmeans.fit(arrond_df)

In [ ]:
interventions_df['cluster'] = kmeans.labels_

In [ ]:
type(unique_incident_types)

In [ ]:
interventions_df = interventions_df[interventions_df['INCIDENT_TYPE_DESC'].isin(unique_incident_types)]

In [ ]:

clusters = interventions_df.groupby(['cluster', 'INCIDENT_TYPE_DESC']).size()
fig2, ax2 = plt.subplots(figsize = (30, 15))
sns.heatmap(clusters.unstack(level = 'INCIDENT_TYPE_DESC'), ax = ax2, cmap = 'Reds')

ax2.set_xlabel('INCIDENT_TYPE_DESC', fontdict = {'weight': 'bold', 'size': 24})
ax2.set_ylabel('cluster', fontdict = {'weight': 'bold', 'size': 24})
for label in ax2.get_xticklabels():
    label.set_size(16)
    label.set_weight("bold")
for label in ax2.get_yticklabels():
    label.set_size(16)
    label.set_weight("bold")

In [ ]:
interventions_df.head(40)